In [1]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import torch, os, re, pandas as pd, json
import torch
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling, DataCollatorWithPadding, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoConfig

import pandas as pd
# from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
# from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
from torch import optim
import gc
from torch.nn import DataParallel
import nltk
import random
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
device = "cuda:7" if torch.cuda.is_available() else "cpu"

In [4]:
print(str(device))

cuda:7


In [5]:
!pwd

/home/ubuntu


In [6]:
with open('/home/ubuntu/Darwin_Gatsby/gatsby_raw.txt', 'r') as file:
    gatsby = file.read().replace('\n', ' ')

with open('/home/ubuntu/Darwin_Gatsby/origin_of_species_raw.txt', 'r') as file:
    origin = file.read().replace('\n', ' ')

In [7]:
gatsby_sen_list = nltk.sent_tokenize(gatsby)
origin_sen_list = nltk.sent_tokenize(origin)
print(len(gatsby_sen_list))
print(len(origin_sen_list))

2439
3981


In [8]:
# ref https://stackoverflow.com/questions/8689795/how-can-i-remove-non-ascii-characters-but-leave-periods-and-spaces
import re
import string
from collections import Counter
from collections import defaultdict
gatsby_first_words_list = []
for sen_g in gatsby_sen_list:
    sen_g = re.sub(r'^\s*|\s\s*', ' ', sen_g).strip().rstrip().lstrip()
    printable = set(string.printable)
    sen_g = filter(lambda x: x in printable, sen_g)
    sen_g = ''.join(filter(lambda x: x in printable, sen_g))
    sen_g_list = sen_g.split()
    #print(sen_g_list[0:3])
    gatsby_first_words_list.append(sen_g_list[0:3])
    
origin_first_words_list = []  
for sen_o in origin_sen_list:
    sen_o = re.sub(r'^\s*|\s\s*', ' ', sen_o).strip().rstrip().lstrip()
    printable = set(string.printable)
    sen_o = filter(lambda x: x in printable, sen_o)
    sen_o = ''.join(filter(lambda x: x in printable, sen_o))
    sen_o_list = sen_o.split()
    # print(sen_o_list[0:3])
    origin_first_words_list.append(sen_o_list[0:3]) 
    
initial_unigrams_gatsby = defaultdict(int)
initial_bigrams_gatsby = defaultdict(int)
initial_trigrams_gatsby = defaultdict(int)
initial_unigrams_origin = defaultdict(int)
initial_bigrams_origin = defaultdict(int)
initial_trigrams_origin = defaultdict(int)
for first_words in gatsby_first_words_list:
    initial_unigrams_gatsby[first_words[0]] += 1
    initial_bigrams_gatsby[' '.join(first_words[:2])] += 1
    initial_trigrams_gatsby[' '.join(first_words)] += 1
    
for first_words in origin_first_words_list:
    initial_unigrams_origin[first_words[0]] += 1
    initial_bigrams_origin[' '.join(first_words[:2])] += 1
    initial_trigrams_origin[' '.join(first_words)] += 1
    
unigram_overlap = set(initial_unigrams_gatsby.keys())
unigram_overlap = unigram_overlap.intersection(initial_unigrams_origin.keys())

bigram_overlap = set(initial_bigrams_gatsby.keys())
bigram_overlap = bigram_overlap.intersection(initial_bigrams_origin.keys())
   
trigram_overlap = set(initial_trigrams_gatsby.keys())
trigram_overlap = trigram_overlap.intersection(initial_trigrams_origin.keys())


# print(f"unigram_overlap: {str(unigram_overlap)}")
# print("\n")
# print(f"bigram_overlap: {str(bigram_overlap)}")
# print("\n")
# print(f"trigram_overlap: {str(trigram_overlap)}")

bigram_list = list(bigram_overlap)
    
# combine the list

# combined_g_o_list = gatsby_first_words_list + origin_first_words_list

# mini_sen_list = []
# for begin in combined_g_o_list:
#     mini_sen_list.append(' '.join(begin))
    
# # ref https://stackoverflow.com/questions/43473736/most-common-2-grams-using-python
# bigrams = zip(mini_sen_list, mini_sen_list[1:])
# counts = Counter(bigrams)
# print(counts.most_common())

In [9]:
# base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [10]:
# head_model = GPT2LMHeadModel.from_pretrained('gpt2-xl').to(device)
# # head_model = GPT2LMHeadModel.from_pretrained('gpt2-xl')
head_model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)
# head_model = GPT2LMHeadModel.from_pretrained('gpt2-xl')
# standard_gpt = GPT2LMHeadModel.from_pretrained('gpt2').to(device)

In [12]:
# model = GPT2LMHeadModel.from_pretrained('gpt2-xl')
# head_model.load_state_dict(torch.load("/home/ubuntu/pos_imdb_model"))
head_model.load_state_dict(torch.load("/home/ubuntu/small_neg_midterm_version"))
# head_model.eval()


<All keys matched successfully>

In [13]:
type(head_model)

transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel

In [14]:
!pwd

/home/ubuntu


In [15]:
import pandas as pd
  
# list of strings
# lst = ['I', 'Movie', 'He', 'What', 
#             'The', 'We', 'I thought', 'I sat', 'How about', 'It was', "This is going", "The nature", ]
  
lst = random.sample(bigram_list,10)
# Calling DataFrame constructor on list
df = pd.DataFrame(lst)
df

,0
0,At a
1,Most of
2,But how
3,By the
4,The eyes
5,After a
6,I could
7,With the
8,It is
9,I have


In [24]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=50, #maximum number of words
    top_p=0.8,
    temperature=.5,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated.to(device), labels=generated.to(device))
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated.to(device), next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().cpu().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
                output_list = list(generated.squeeze().cpu().numpy())
                # output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
                output_text = f"{tokenizer.decode(output_list)}" 
                generated_list.append(output_text)
                
    return generated_list

#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data, model, tokenizer):
    generated_lyrics = []
    for i in range(len(test_data)):
        x = generate(model, tokenizer, test_data[0][i], entry_count=1)
        generated_lyrics.append(x)
    return generated_lyrics

#Run the functions to generate the lyrics
generated_text = text_generation(df, head_model, base_tokenizer)

100%|██████████| 1/1 [00:00<00:00,  2.74it/s]


In [25]:
for text in generated_text:
    print(text, "\n")

['At a minimum, the three-year minimum wage is $12.25 an hour, and the minimum wage for a full-time worker is $12.50. The minimum wage for a full-time worker is $12.50. The minimum wage'] 

['Most of the people who were in the hospital were in shock and had no idea what was going on. The doctor told me that the patient had been taken to the hospital for surgery, but had not been able to see any of the other patients. He said'] 

['But how much of this is true? Why is it that the "evidence" that the "evidence" that the "evidence" is "valid" is so weak? Why is it that the "evidence" that the "evidence" is "valid" is'] 

['By the time the game was released, the story was already in the public domain. The game was released on the PC, but the PC version was not released until the end of the year. The game was released on the PC, but the PC version was'] 

['The eyes of the people who were watching, and the people who were watching, and the people who were watching, and the people who were w